In [1]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.preprocessing import MinMaxScaler
from src.twitter_interface import TwitterInterface
from src.asset_selector import AssetSelector
from src.indicator_collection import Indicators
from src.finta_interface import Indicator
from util import time_formatter
from src.predictor import Predictor
from sklearn.cluster import KMeans
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import pandas as pd
import numpy as np
import configparser
import requests
import twitter
import json
import time
import sys
import os

weeks = 54

backdate = time_formatter(time.time() - (604800 * weeks))
config = configparser.ConfigParser()
try:
    config.read(os.path.relpath("../config.ini"))
except FileExistsError as e:
    print("FileExistsError: {}".format(e))
    sys.exit(1)
alpaca_api = tradeapi.REST(
    base_url    = config["alpaca"]["APCA_API_BASE_URL"],
    key_id      = config["alpaca"]["APCA_API_KEY_ID"],
    secret_key  = config["alpaca"]["APCA_API_SECRET_KEY"],
    api_version = config["alpaca"]["VERSION"]
)
twitter_api = twitter.Api(
    config["twitter"]["CONSUMER_KEY"],
    config["twitter"]["CONSUMER_SECRET"],
    config["twitter"]["ACCESS_TOKEN_KEY"],
    config["twitter"]["ACCESS_TOKEN_SECRET"]
)
tq_key = config["tenquant"]["api_key"]

tdf = pd.DataFrame()
ti = TwitterInterface(twitter_api, tdf)
trading_account = alpaca_api.get_account()

selector = AssetSelector(alpaca_api, edgar_token=None)
indicators = Indicators(alpaca_api)

def get_sentiment(text):
    """Given a text block, return a sentiment score based.
    :param text:
    :return:
    """
    text_polarity   = sid.polarity_scores(text)

    if text_polarity["compound"] > 0.50:
        sentiment   = "positive"
    else:
        sentiment   = "negative"
    return sentiment, text_polarity["compound"]


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ben/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
pool = 3
# gainers = selector.top_gainers()
# print()
# losers = selector.top_losers()
print()
bulls = selector.bullish_candlesticks(64, pool)
# print()
# bears = selector.bearish_candlesticks(64, pool)


                    Bulls                    

Ticker     Last       Change     % Change   MACD?      MFI?       VZO?       Stoch?     Pattern
––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––


sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...


GCI        $6.00      $-0.08     -0.04%     True       False      False      True       inverseHammer


sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...


CSII       $45.67     $1.49      0.03%      True       False      False      True       hammer


sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v1/bars/1D 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.marke

MRAM       $5.39      $0.09      -0.04%     True       False      True       True       inverseHammer


In [3]:
ticker = "GCI"

In [4]:
sid = SentimentIntensityAnalyzer()
stocktwits = requests.post("https://api.stocktwits.com/api/2/streams/symbol/{}.json".format(ticker))
stocktwits = json.loads(stocktwits.text)
stocktwitstext = "\n".join([m["body"] for m in stocktwits["messages"]])
tweets = ti.get_tweets(ticker, "stock")

AttributeError: 'str' object has no attribute 'GetSearch'

In [5]:
stsentiment, stscore = get_sentiment(stocktwitstext)
tsentiment, tscore = get_sentiment(tweets)

NameError: name 'tweets' is not defined

In [6]:
print("Sentiment score for {}".format(ticker))
print("{:<30}".format("–" * 45))
print("Stocktwits:\t{}\t score: {}\nTwitter:\t{}\t score:\t{}".format(stsentiment, stscore, tsentiment, tscore))
print("Average:\t{}".format((stscore + tscore)/2))

Sentiment score for GCI
–––––––––––––––––––––––––––––––––––––––––––––


NameError: name 'tsentiment' is not defined

In [7]:
tenquant_data = requests.get("https://api.tenquant.io/data?key={}&ticker={}".format(tq_key, ticker))
tenquant_data = json.loads(tenquant_data.text)

In [8]:
stocktwits

{'response': {'status': 200},
 'symbol': {'id': 5469,
  'symbol': 'GCI',
  'title': 'Gannett Co., Inc.',
  'aliases': [],
  'is_following': False,
  'watchlist_count': 584},
 'cursor': {'more': True, 'since': 186851859, 'max': 186227998},
 'messages': [{'id': 186851859,
   'body': '$GCI can we hit 6.10 tday?',
   'created_at': '2019-12-13T15:28:21Z',
   'user': {'id': 1267383,
    'username': 'oeni',
    'name': 'Oeni',
    'avatar_url': 'https://avatars.stocktwits.com/production/1267383/thumb-1510679628.png',
    'avatar_url_ssl': 'https://avatars.stocktwits.com/production/1267383/thumb-1510679628.png',
    'join_date': '2017-11-14',
    'official': False,
    'identity': 'User',
    'classification': [],
    'followers': 5,
    'following': 0,
    'ideas': 757,
    'watchlist_stocks_count': 17,
    'like_count': 10,
    'plus_tier': '',
    'premium_room': ''},
   'source': {'id': 1149,
    'title': 'StockTwits for iOS',
    'url': 'http://www.stocktwits.com/mobile'},
   'symbols': [

In [ ]:
tweets 

In [ ]:
tenquant_data